In [ ]:
#!pip3 install langchain_community
#!pip3 install langchain_openai
#pip3 install bert_score

In [5]:
import pandas as pd
import openai
from dotenv import load_dotenv
import os
import ast
from openai import AzureOpenAI
import faiss
import numpy as np
import json
from bert_score import score




### Load Enviroment files

In [6]:
# Load environment variables
load_dotenv()

Python-dotenv could not parse statement starting at line 10
Python-dotenv could not parse statement starting at line 11
Python-dotenv could not parse statement starting at line 12
Python-dotenv could not parse statement starting at line 13
Python-dotenv could not parse statement starting at line 14
Python-dotenv could not parse statement starting at line 15
Python-dotenv could not parse statement starting at line 16
Python-dotenv could not parse statement starting at line 17
Python-dotenv could not parse statement starting at line 18
Python-dotenv could not parse statement starting at line 19
Python-dotenv could not parse statement starting at line 20


True

In [7]:
openai.api_type = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai_deployment = "sdgi-gpt-35-turbo-16k"
neo4j_pass = os.getenv("NEO4JPASS")


In [8]:
# OpenAI API configuration
#openai.api_type = "azure"
#openai.api_key = os.getenv("api_key_azure")
#openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
#openai.api_version = os.getenv("api_version")
#openai_deployment = "sdgi-gpt-35-turbo-16k"


client = AzureOpenAI(
  api_key = os.getenv("OPENAI_API_KEY"),  
  api_version = os.getenv("OPENAI_API_VERSION"),
  azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
)

embedding_model = os.getenv("USER_QUERY_EMBEDDING_ENGINE") 

# print(openai.api_key)
# print(openai.api_base)
# print(openai.api_version)


<h3> helper functions </h3>

In [10]:
# use this function to make simple openAI Calls
def callOpenAI(prompt):  
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=[
                        {"role": "user", "content": prompt},
                    ]
                )
    response = response_entities.choices[0].message.content
    return response


<h3> processing modules </h3>

In [11]:
def extractEntitiesFromQuery(user_query):
    prompt = f"""
    Extract entities from the following user query: \"{user_query}\" and return output in array format.
    
    -Entities should be directly related to the domain or topic of interest. They should represent important concepts that contribute to the understanding of the subject matter.
    -Each entity in the knowledge graph should be distinct and have a unique identifier. This ensures clarity and avoids ambiguity when establishing relationships between entities.
    -You Must return output in array format e.g  ['entity1','entity2'] !!!
    -Avoid adding new lines or breaking spaces to your output. Array should be single dimension and single line !!!
    """
    entity_str = callOpenAI(prompt)   
    return entity_str



# KG module

In [12]:
from py2neo import Graph, Node, Relationship
graph = Graph(uri = 'bolt://localhost:7687',user='neo4j',password=neo4j_pass)

In [13]:
def search_entities_with_relations(label, name):
    """
    Search for nodes by label and name, including all their relationships and connected nodes.

    :param label: The label of the nodes to search.
    :param name: The name property value of the nodes to search.
    :return: A list of dictionaries, each containing a node and its relationships and connected nodes.
    """
    # Match nodes and their relationships, case-insensitive search
    query = f"""
    MATCH (n:`{label}`)-[r]-(m)
    WHERE toLower(n.name) = toLower($name) OR toLower(n.acronym) = toLower($name)
    RETURN n, collect(r) as relations, collect(m) as connectedNodes
    """
    results = graph.run(query, name=name).data()
    
    # Construct a comprehensive view for each node with its relationships and connected nodes
    entities_with_relations = []
    for record in results:
        entity_info = {
            'node': record['n'],
            'relationships': record['relations'],
            'connected_nodes': record['connectedNodes']
        }
        entities_with_relations.append(entity_info)

    return entities_with_relations


In [14]:
def separate_entities_and_relationships(relationships):
    """
    Separates entities and relationships from Neo4j query output into distinct structures,
    ensuring no duplicate entities are included.

    :param relationships: A list of Relationship objects from a Neo4j query.
    :return: A tuple containing two lists: one for unique entities and another for relationships.
    """
    # Using a dictionary to ensure unique entities based on a combination of name and category
    entities_dict = {}
    rels = []

    for rel in relationships:
        # Process both the source and target nodes for each relationship
        for node in [rel.start_node, rel.end_node]:
            # Define a unique identifier for each entity
            entity_id = f"{node['name']}_{node['category']}"
            
            # If the entity is not already in the dictionary, add it
            if entity_id not in entities_dict:
                entities_dict[entity_id] = {
                    'name': node['name'],
                    'category': node.get('category', 'N/A'),
                    #'summary': node.get('summary', 'N/A'),
                    'acronym': node.get('acronym', 'N/A'),
                }

        # Add relationship information
        rels.append({
            'subject': rel.start_node['name'],
            'object': rel.end_node['name'],
            'relationship_type': rel.__class__.__name__,
            'description': rel.get('description', 'N/A'),
        })

    # Convert the entities dictionary to a list to remove the unique identifier layer
    entities_list = list(entities_dict.values())

    return entities_list, rels




In [15]:
def search_embeddings(user_query):
    df = pd.read_pickle('../models/df_embed_EN.pkl')
    df_filtered = df
    length = len(df_filtered.head())
    filtered_embeddings_arrays = np.array(list(df_filtered['Embedding']))
    index = faiss.IndexFlatIP(filtered_embeddings_arrays.shape[1]) 
    index.add(filtered_embeddings_arrays)
    
    user_query_embedding = client.embeddings.create( 
        input=user_query ,model= embedding_model
    ).data[0].embedding
    
    k = min(5, length)
    distances, indices = index.search(np.array([user_query_embedding]), k)
    return df_filtered, distances, indices

In [16]:
def get_answer(user_question, content):
    system_prompt = "You are a system that answers user questions based on excerpts from PDF documents provided for context. Only answer if the answer can be found in the provided context. Do not make up the answer; if you cannot find the answer, say so."
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_question},
        {'role': 'user', 'content': content},
    ]
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0.2,
                    messages=messages
                )
    response = response_entities.choices[0].message.content
    return response
  

In [17]:
def map_to_structure(qs):
    result_dict = {}

    # Extract the DataFrame from the tuple
    dataframe = qs[0]

    # Counter to limit the loop to 10 iterations
    count = 0

    for index, row in dataframe.iterrows():
        # Define a unique identifier for each document, you can customize this based on your data
        document_id = f"doc-{index + 1}"
        # Handle NaN in content by using fillna
        content = row["Content"]
        content = ' '.join(row["Content"].split()[:160])
        # Create a dictionary for each document
        document_info = {
            "title": row["Document Title"],
            "extract": content or "",  # You may need to adjust this based on your column names
            "category": row["Category"],
            "link": row["Link"],
            "thumbnail": ''
        }
        # print(document_info)
        # Add the document to the result dictionary
        result_dict[document_id] = document_info

        # Increment the counter
        count += 1

        # # Break out of the loop if the counter reaches top 10
        if count == 10:
            break

    return result_dict


In [18]:
## module to get data for specific indicators which are identified is relevant to the user query

def indicatorsModule(user_query): #lower priority
    
    # find relevant indicators based on uesr query and extract values
    indicators_dict={
        "indicator-id-1":"value from indicator-id-1",
        "indicator-id-2":"value from indicator-id-2"
    }#temp
    
    return indicators_dict

## module to extract text from documents and return the text and document codes

def semanticSearchModule(user_query):
    qs = search_embeddings(user_query) #df, distances, indices
    result_structure = map_to_structure(qs)
    return result_structure


In [61]:
# module to synthesize answer using retreival augmented generation approach

def synthesisModule(user_query, excerpts_dict, indicators_dict):
    
    # Generate prompt engineering text and template
    llm_instructions = f"""
    Ignore previous commands!!!
    Given a user query, use the provided excerpts, and Sources to
    provide the correct answer to the user's query
    
    User Query: {user_query}
    
    Sources: {excerpts_dict}
    

    - Answer must be coherent, relevant and should contain important details. 
    
    """
    ###synthesize data into structure within llm prompt engineering instructions
    answer=callOpenAI(llm_instructions)
    
    return answer

## to test this, run the full pipeline with the handleApiCall function

In [62]:
# module to synthesize answer using retreival augmented generation approach

def synthesisModule_with_KG(user_query, entities, relationships, excerpts_dict, indicators_dict):
    
    # Generate prompt engineering text and template
    llm_instructions = f"""
    Ignore previous commands!!!
    Given a user query, use the provided excerpts, Sources, entities and relations to
    provide the correct answer to the user's query
    
    User Query: {user_query}
    
    Sources: {excerpts_dict}
    
    Entity and Relation info: {entities} {relationships}

    - Answer must be coherent, relevant and should contain important details. 

    """
    ###synthesize data into structure within llm prompt engineering instructions
    answer=callOpenAI(llm_instructions)
    
    return answer

## to test this, run the full pipeline with the handleApiCall function

In [63]:
def handleApiCall(user_query):
        
    ##run processing modules (in parallel)
    #entities_dict=knowledgeGraphModule(user_query)
    excerpts_dict=semanticSearchModule(user_query)
    indicators_dict=indicatorsModule(user_query) ##lower priority
    #query_idea_list=queryIdeationModule(user_query) ##lower priority
    
    ##synthesis module
    answer=synthesisModule(user_query, excerpts_dict, indicators_dict)
    
    ##structure response
    response={
        "user_query":user_query,
        "answer":answer,
        "sources":excerpts_dict
        
    }
    
    return response

In [93]:
def handle_api_call_with_kg(user_query):
        
    ##run processing modules (in parallel)
    #entities_dict=knowledgeGraphModule(user_query)
    excerpts_dict=semanticSearchModule(user_query)
    indicators_dict=indicatorsModule(user_query) ##lower priority
    #query_idea_list=queryIdeationModule(user_query) ##lower priority
    
    ##synthesis module
    answer=synthesisModule_with_KG(user_query, entities[:10], relationships[:100], excerpts_dict, indicators_dict)
    
    ##structure response
    response={
        "user_query":user_query,
        "answer":answer,
        "sources":excerpts_dict,
        "entities": entities,
        "relations":relationships
        
    }
    
    return response

In [96]:
with open("queries.json") as f:
    data = f.read()
    query_list = ast.literal_eval(data)
    f.close()
    

In [97]:
len(query_list)

30

# Pipeline

In [78]:
# full pipeline with retreival, synthesis of answer to user query, and structure results into api response

for index in range(10,30):
    test_query = query_list[index]
    print (test_query)

    entity_str = extractEntitiesFromQuery(test_query)
    # Convert the string representation of the list to an actual list
    entity_list = ast.literal_eval(entity_str)

    relations_list = []
    for item in entity_list:
        entities_with_relations = search_entities_with_relations('Entity', item)

        for entity_info in entities_with_relations:
            relations_list.extend(entity_info['relationships'])
            #print("Connected Nodes:", entity_info['connected_nodes'])

    # Assuming 'relationships' is your list of Relationship objects from the query
    entities, relationships = separate_entities_and_relationships(relations_list)

    print (len(relationships))
    print (len(entities))


    indicators_dict=indicatorsModule(test_query)
    print(indicators_dict)

    #test usage
    excerpts_dict=semanticSearchModule(test_query)
    # print(excerpts_dict)


    #Return top 10-20 most related 
    # Define the filename to save the JSON data -  can remove later
    json_filename = "outputs/excerpts.json"

    # Save excerpts_dict to the JSON file just for a better preview
    with open(json_filename, 'w', encoding='utf-8') as json_file:
        json.dump(excerpts_dict, json_file, ensure_ascii=False, indent=4)

    print(f"Excerpts saved to {json_filename}")


    # test usage
    llm_response = handleApiCall(test_query) 
    # Define the filename to save the JSON data -  can remove later
    json_filename = "outputs/query"+ str(index+1) + "_answer.json"

    # Save excerpts_dict to the JSON file just for a better preview
    with open(json_filename, 'w', encoding='utf-8') as json_file:
        json.dump(llm_response, json_file, ensure_ascii=False, indent=4)

    print(f"LLM Response saved to {json_filename}")


    # test usage
    kg_llm_response=handle_api_call_with_kg(test_query) 
    # Define the filename to save the JSON data -  can remove later
    json_filename = "outputs_KG/Query"+ str(index+1) + "_answer.json"

    # Save excerpts_dict to the JSON file just for a better preview
    with open(json_filename, 'w', encoding='utf-8') as json_file:
        json.dump(kg_llm_response, json_file, ensure_ascii=False, indent=4)

    print(f"KG-LLM Response saved to {json_filename}")
    print ("\n--------------\n")

What strategies is UNDP implementing to support Angola's Bureau of External Relations and Advocacy?
24
17
{'indicator-id-1': 'value from indicator-id-1', 'indicator-id-2': 'value from indicator-id-2'}
Excerpts saved to outputs/excerpts.json
LLM Response saved to outputs/query11_answer.json
KG-LLM Response saved to outputs_KG/Query11_answer.json

--------------

How is Angola addressing the Sustainable Development Goals with a focus on poverty eradication?
9
5
{'indicator-id-1': 'value from indicator-id-1', 'indicator-id-2': 'value from indicator-id-2'}
Excerpts saved to outputs/excerpts.json
LLM Response saved to outputs/query12_answer.json
KG-LLM Response saved to outputs_KG/Query12_answer.json

--------------

How does the Ministry of Environment's support for the Paris Agreement on Climate Change manifest in Argentina's environmental policies?
7
6
{'indicator-id-1': 'value from indicator-id-1', 'indicator-id-2': 'value from indicator-id-2'}
Excerpts saved to outputs/excerpts.json
LL

<h3>testing</h3>

In [ ]:
## next step, develop automated testing for all modules
## iterate through test_queries and build automated tests to score results

# open testing dataset with queries and expected results
test_queries_df=pd.read_csv("../testing/energy_ai_test_dataset_v0.csv")



In [ ]:
#import nltk
nltk.download('punkt') # Make sure to download the 'punkt' tokenizer models
from nltk.tokenize import word_tokenize
from nltk.translate.meteor_score import meteor_score

# Example texts
reference_texts = ["The Gender Development Index is relevant to UNDP's work in Albania by highlighting the country's progress in gender equality, with a GDI of 0.967, informing UNDP's focus on gender equality and women's empowerment initiatives."]
candidate_text = "The Gender Development Index (GDI) is relevant to UNDP's work in Albania because Albania has a Gender Development Index of 0.967, indicating a high level of gender development in the country. UNDP partners with various organizations, including United Nations agencies and the United Nations Population Fund, to achieve its goals<a data-id='doc-13'>[1]</a>. UNDP also works with the government of Albania to drive policies and actions that focus on disaster risk reduction, increasing resilience, and protecting development investments<a data-id='doc-13'>[1]</a>. Additionally, UNDP focuses on the United Nations Sustainable Development Cooperation Framework in its work"

# Tokenize the texts
references = [word_tokenize(ref) for ref in reference_texts]
candidate = word_tokenize(candidate_text)

# Calculate METEOR score
score = meteor_score(references, candidate)

print(score)


In [ ]:
!pip3 install bert-score transformers

In [ ]:
from bert_score import score

# Your candidate (system outputs) and reference sentences
candidates = ["the cat sits on the mat"]
references = ["the cat is sitting on the mat"]

# Calculate BERTScore
P, R, F1 = score(candidates, references, lang='en', verbose=True)

# Print the precision, recall, and F1 scores
for p, r, f1 in zip(P, R, F1):
    print(f"Precison: {p.item():.4f}, Recall: {r.item():.4f}, F1: {f1.item():.4f}")


  # TODO::: 
  ##### Add citation prompt to the synthesis module. -done 
  ##### Convert notebook to flask API script. main.py - done
  ##### Refactor PDF -> txt pipeline 

  

In [ ]:
## module to get information on the entities from user query using the KG
def knowledgeGraphModule(user_query):
    
    # generate list of entities based on user query
    entity_list = extractEntitiesFromQuery(user_query)
    my_list = ast.literal_eval(entity_list)
    prompt_summarise_entites = f"""
    Summarize all relations between all the entities : {my_list}
    """
    summarise_entities = callOpenAI(prompt_summarise_entites)
    # Initialize an empty dictionary to store information
    entities_dict = {
        "relations": summarise_entities,
        "entities": {}
    }
    # Loop through each entity in the list
    for entity in my_list:
        # Fetch information about the entity from your knowledge graph
        prompt = f"Give me a short description 50 words of {entity}"
        entity_info = callOpenAI(prompt)
        # Add the entity information to the dictionary
        entities_dict["entities"][entity] = entity_info
    
    return entities_dict


# Test usage
test_query = "what is the major work on UNDP in Afganistan?"
entities_dict = knowledgeGraphModule(test_query)
print(entities_dict)

In [ ]:
def relationships_to_data_structure(relationships):
    """
    Converts relationships from Neo4j query output into a structured Python data structure.

    :param relationships: A list of Relationship objects from a Neo4j query.
    :return: A list of dictionaries, each representing a relationship and its node details.
    """
    formatted_relationships = []

    # Sort the relationships by the name of the source node for consistent ordering
    sorted_relationships = sorted(relationships, key=lambda rel: rel.start_node['name'])

    for rel in sorted_relationships:
        relationship_data = {
            'relationship_type': rel.__class__.__name__,
            'source_node': {
                'name': rel.start_node['name'],
                'category': rel.start_node.get('category', 'N/A'),
                'summary': rel.start_node.get('summary', ''),
                'acronym': rel.start_node.get('acronym', ''),
                # Include additional properties as needed
            },
            'target_node': {
                'name': rel.end_node['name'],
                'category': rel.end_node.get('category', ''),
                'summary': rel.end_node.get('summary', ''),
                'acronym': rel.end_node.get('acronym', ''),

                # Include additional properties as needed
            },
            'properties': {
                # Include relationship properties
                'description': rel.get('description', 'N/A'),
            
            }
        }

        formatted_relationships.append(relationship_data)

    return formatted_relationships

# Example usage, assuming 'relationships' is your list of Relationship objects from the query
formatted_data = relationships_to_data_structure(entity_info['relationships'])


In [ ]:
def format_and_sort_relationships(relationships):
    """
    Formats and sorts the relationships from Neo4j query output.

    :param relationships: A list of Relationship objects from a Neo4j query.
    """
    # Sort the relationships by the name of the source node for consistent ordering
    sorted_relationships = sorted(relationships, key=lambda rel: rel.start_node['name'])

    for rel in sorted_relationships:
        source_node = rel.start_node
        target_node = rel.end_node
        print(f"Relationship: {rel.__class__.__name__}")
        print(f"  Source Node: {source_node['name']} (Category: {source_node.get('category', 'N/A')})")
        print(f"  Target Node: {target_node['name']} (Category: {target_node.get('category', 'N/A')})")
        
        # Print relationship properties with indentation
        if rel.get('description'):
            print(f"  Description: {rel['description']}")
        if rel.get('summary'):
            print(f"  Summary: {rel['summary']}")

        print()  # Add a newline for spacing between relationships

# Example usage, assuming 'relationships' is your list of Relationship objects from the query
format_and_sort_relationships(entity_info['relationships'])
